# A Movie Database
### Data Engineering Capstone Project

#### Project Summary
This project makes use of the movie data from approximately 57,000 movies listed on TMDB.org to implement a Snowflake schema which includes data on each movie (obtained via a priori API query to create JSON files which include details such as the top-credited actors, director(s) and writer(s)), IMDB average ratings, and additional data from another set of users who have rated and/or tagged these movies independent of IMDB.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import math

import os
import requests

import datetime
import time
import json

import psycopg2
from psycopg2.extras import execute_values
from psycopg2.extensions import register_adapter, AsIs

from sql_queries import *

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>


#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The data includes some of the CSV files obtained from the University of Minnesota's MovieLens Dataset (https://grouplens.org/datasets/movielens/latest/):

##### 1) links.csv - 58,098 rows linking the movieId in movies, ratings & tags CSV files (below) to the tmdbId for the same movie

In [2]:
links = pd.read_csv('datasets/links.csv')

print(links.info())
print(links.shape)
links.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
imdbId     58098 non-null int64
tmdbId     57917 non-null float64
dtypes: float64(1), int64(2)
memory usage: 1.3 MB
None
(58098, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


##### 2) movies.csv - 58,098 rows listing the movies represented in the ratings & tags CSVs (below)

In [3]:
movies = pd.read_csv('datasets/movies.csv')

print(movies.info())
print(movies.shape)
movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB
None
(58098, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


##### 3) ratings.csv - 10,747,027 timestamped user ratings (on a 0-5 scale) for the movies above

In [4]:
ratings = pd.read_csv('datasets/ratings.csv')

print(ratings.info())
print(ratings.shape)
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10747027 entries, 0 to 10747026
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 328.0 MB
None
(10747027, 4)


,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


##### 4) tags.csv - 1,108,997 timestamped user tags (on a 0-5 scale) for the movies above

In [5]:
tags = pd.read_csv('datasets/tags.csv')

print(tags.info())
print(tags.shape)
tags.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
userId       1108997 non-null int64
movieId      1108997 non-null int64
tag          1108981 non-null object
timestamp    1108997 non-null int64
dtypes: int64(3), object(1)
memory usage: 33.8+ MB
None
(1108997, 4)


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


##### 5) people.json - 57,160 rows; TMDB API responses to credits (top cast, directors, writers) queries for titles in movies

In [6]:
people = pd.read_json('datasets/people.json')

print(people.info())
print(people.shape)
people.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57160 entries, 0 to 9999
Data columns (total 16 columns):
id              57160 non-null int64
actor1_id       55082 non-null float64
actor2_id       53460 non-null float64
actor3_id       52462 non-null float64
actor4_id       51496 non-null float64
actor5_id       49761 non-null float64
director1_id    56705 non-null float64
director2_id    4007 non-null float64
director3_id    543 non-null float64
director4_id    240 non-null float64
director5_id    164 non-null float64
writer1_id      48257 non-null float64
writer2_id      20663 non-null float64
writer3_id      6130 non-null float64
writer4_id      1838 non-null float64
writer5_id      586 non-null float64
dtypes: float64(15), int64(1)
memory usage: 7.4 MB
None
(57160, 16)


,id,actor1_id,actor2_id,actor3_id,actor4_id,actor5_id,director1_id,director2_id,director3_id,director4_id,director5_id,writer1_id,writer2_id,writer3_id,writer4_id,writer5_id
0,862,31.0,12898.0,7167.0,12899.0,12900.0,7879.0,NaN,NaN,NaN,NaN,7.0,12891.0,12892.0,12893.0,NaN
1,8844,2157.0,205.0,145151.0,5149.0,8537.0,4945.0,NaN,NaN,NaN,NaN,876.0,56520.0,56521.0,NaN,NaN
10,9087,3392.0,516.0,8349.0,19839.0,20767.0,3026.0,NaN,NaN,NaN,NaN,13520.0,NaN,NaN,NaN,NaN
100,47475,14.0,8984.0,3234.0,923.0,23709.0,14692.0,NaN,NaN,NaN,NaN,18382.0,1236253.0,2675240.0,NaN,NaN
1000,9078,21877.0,12158.0,57313.0,7139.0,159552.0,57314.0,NaN,NaN,NaN,NaN,57316.0,NaN,NaN,NaN,NaN


##### 6) details.json - 57,160 rows; TMDB API responses to additional movie details queries for titles in movies

In [7]:
details = pd.read_json('datasets/details.json')

print(details.info())
print(details.shape)
details.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57160 entries, 0 to 57159
Data columns (total 25 columns):
adult                    57160 non-null bool
backdrop_path            41470 non-null object
belongs_to_collection    6020 non-null object
budget                   57160 non-null int64
genres                   57160 non-null object
homepage                 56526 non-null object
id                       57160 non-null int64
imdb_id                  57160 non-null object
original_language        57160 non-null object
original_title           57160 non-null object
overview                 57160 non-null object
popularity               57160 non-null float64
poster_path              54183 non-null object
production_companies     57160 non-null object
production_countries     57160 non-null object
release_date             57160 non-null object
revenue                  57160 non-null int64
runtime                  57072 non-null float64
spoken_languages         57160 non-null object
st

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/XLwjO1NSCIaLznh58OQtmSFl0N.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033,81.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Toy Story,False,7.9,14111
1,False,/6w31RRm2s2CK1r3xDLf12WgIaHa.jpg,"{'id': 495527, 'name': 'Jumanji Collection', '...",65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.sonypictures.com/movies/jumanji/,8844,tt0113497,en,Jumanji,...,1995-12-15,262797249,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Roll the dice and unleash the excitement!,Jumanji,False,7.2,8260
2,False,/nh9gYaXHTNT9yylX10L9aGqFehy.jpg,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602,tt0113228,en,Grumpier Old Men,...,1995-12-22,0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,251
3,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156,127.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.3,95
4,False,/1XUPR3Ki1fvZDCtetcepMoz7oqu.jpg,"{'id': 96871, 'name': 'Father of the Bride (St...",0,"[{'id': 35, 'name': 'Comedy'}]",,11862,tt0113041,en,Father of the Bride Part II,...,1995-12-08,76594107,106.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,6.2,495


##### 7) actors.json - 93,346 rows; TMDB API responses to actor details queries for the top cast in movies

In [8]:
actors_df = pd.read_json('datasets/actors.json')

print(actors_df.info())
print(actors_df.shape)
actors_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93346 entries, 0 to 93345
Data columns (total 17 columns):
adult                   93344 non-null float64
also_known_as           93344 non-null object
biography               93344 non-null object
birthday                49720 non-null object
deathday                14016 non-null object
gender                  93344 non-null float64
homepage                6407 non-null object
id                      93344 non-null float64
imdb_id                 81165 non-null object
known_for_department    93344 non-null object
name                    93344 non-null object
place_of_birth          45133 non-null object
popularity              93344 non-null float64
profile_path            53968 non-null object
status_code             2 non-null float64
status_message          2 non-null object
success                 2 non-null float64
dtypes: float64(6), object(11)
memory usage: 12.1+ MB
None
(93346, 17)


,adult,also_known_as,biography,birthday,deathday,gender,homepage,id,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path,status_code,status_message,success
0,0.0,"[George Walton Lucas Jr. , 乔治·卢卡斯, Джордж Лука...","George Walton Lucas Jr. (born May 14, 1944) is...",1944-05-14,None,2.0,None,1.0,nm0000184,Directing,George Lucas,"Modesto, California, USA",8.257,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg,NaN,NaN,NaN
1,0.0,"[Mark Hamil, Mark Richard Hamill, Марк Хэмилл,...","Mark Richard Hamill (born September 25, 1951) ...",1951-09-25,None,2.0,None,2.0,nm0000434,Acting,Mark Hamill,"Concord, California, USA",8.961,/zMQ93JTLW8KxusKhOlHFZhih3YQ.jpg,NaN,NaN,NaN
2,0.0,"[Гаррісон Форд, Харрисон Форд, هاريسون فورد, 해...",Legendary Hollywood Icon Harrison Ford was bor...,1942-07-13,None,2.0,None,3.0,nm0000148,Acting,Harrison Ford,"Chicago, Illinois, USA",8.641,/5M7oN3sznp99hWYQ9sX0xheswWX.jpg,NaN,NaN,NaN
3,0.0,"[Carrie Frances Fisher , Кэрри Фишер, Кэрри Фр...",Carrie Frances Fisher (21 October 1956 - 27 De...,1956-10-21,2016-12-27,1.0,https://carriefisher.com/,4.0,nm0000402,Acting,Carrie Fisher,"Beverly Hills, Los Angeles, California, USA",3.679,/rfJtncHewKVnHjqpIZvjn24ESeC.jpg,NaN,NaN,NaN
4,0.0,[Peter Wilton Cushing],"Peter Wilton Cushing, OBE (26 May 1913 – 11 A...",1913-05-26,1994-08-11,2.0,None,5.0,nm0001088,Acting,Peter Cushing,"Kenley, Surrey, England, UK",5.950,/if5g03wn6uvHx7F6FxXHLebKc0q.jpg,NaN,NaN,NaN


##### 8) directors.json - 24,223 rows; TMDB API responses to director details queries for directors in movies

In [9]:
directors_df = pd.read_json('datasets/directors.json')

print(directors_df.info())
print(directors_df.shape)
directors_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24223 entries, 0 to 24222
Data columns (total 14 columns):
adult                   24223 non-null bool
also_known_as           24223 non-null object
biography               24223 non-null object
birthday                10473 non-null object
deathday                2865 non-null object
gender                  24223 non-null int64
homepage                1306 non-null object
id                      24223 non-null int64
imdb_id                 21673 non-null object
known_for_department    24223 non-null object
name                    24223 non-null object
place_of_birth          9890 non-null object
popularity              24223 non-null float64
profile_path            8599 non-null object
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 2.4+ MB
None
(24223, 14)


,adult,also_known_as,biography,birthday,deathday,gender,homepage,id,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path
0,False,"[George Walton Lucas Jr. , 乔治·卢卡斯, Джордж Лука...","George Walton Lucas Jr. (born May 14, 1944) is...",1944-05-14,None,2,None,1,nm0000184,Directing,George Lucas,"Modesto, California, USA",8.257,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg
1,False,"[Mark Hamil, Mark Richard Hamill, Марк Хэмилл,...","Mark Richard Hamill (born September 25, 1951) ...",1951-09-25,None,2,None,2,nm0000434,Acting,Mark Hamill,"Concord, California, USA",8.961,/zMQ93JTLW8KxusKhOlHFZhih3YQ.jpg
2,False,[Andrew A. Stanton],"Andrew Stanton (born December 3, 1965) is an A...",1965-12-03,None,2,None,7,nm0004056,Writing,Andrew Stanton,"Boston, Massachusetts, USA",5.120,/tRwWuo06aN9vuXAPaswMN42x2ii.jpg
3,False,[],Lee Unkrich is an American director and film e...,1967-08-08,None,2,None,8,nm0881279,Directing,Lee Unkrich,"Cleveland, Ohio, USA",1.400,/oeUkLlak2lqKRYZmJZZv5gI87Ok.jpg
4,False,[],Albert Lawrence Brooks (born Albert Lawrence E...,1947-07-22,None,2,http://www.albertbrooks.com/,13,nm0000983,Acting,Albert Brooks,"Beverly Hills, California, USA",3.665,/8iDSGu5l93N7benjf6b3AysBore.jpg


##### 9) writers.json - 35,585 rows; TMDB API responses to director details queries for directors in movies

In [10]:
writers_df = pd.read_json('datasets/writers.json')

print(writers_df.info())
print(writers_df.shape)
writers_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35585 entries, 0 to 35584
Data columns (total 17 columns):
adult                   35584 non-null float64
also_known_as           35584 non-null object
biography               35584 non-null object
birthday                12313 non-null object
deathday                4009 non-null object
gender                  35584 non-null float64
homepage                1439 non-null object
id                      35584 non-null float64
imdb_id                 30490 non-null object
known_for_department    35584 non-null object
name                    35584 non-null object
place_of_birth          11167 non-null object
popularity              35584 non-null float64
profile_path            9504 non-null object
status_code             1 non-null float64
status_message          1 non-null object
success                 1 non-null float64
dtypes: float64(6), object(11)
memory usage: 4.6+ MB
None
(35585, 17)


,adult,also_known_as,biography,birthday,deathday,gender,homepage,id,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path,status_code,status_message,success
0,0.0,"[George Walton Lucas Jr. , 乔治·卢卡斯, Джордж Лука...","George Walton Lucas Jr. (born May 14, 1944) is...",1944-05-14,None,2.0,None,1.0,nm0000184,Directing,George Lucas,"Modesto, California, USA",7.677,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg,NaN,NaN,NaN
1,0.0,"[Carrie Frances Fisher , Кэрри Фишер, Кэрри Фр...",Carrie Frances Fisher (21 October 1956 - 27 De...,1956-10-21,2016-12-27,1.0,https://carriefisher.com/,4.0,nm0000402,Acting,Carrie Fisher,"Beverly Hills, Los Angeles, California, USA",3.679,/rfJtncHewKVnHjqpIZvjn24ESeC.jpg,NaN,NaN,NaN
2,0.0,[Andrew A. Stanton],"Andrew Stanton (born December 3, 1965) is an A...",1965-12-03,None,2.0,None,7.0,nm0004056,Writing,Andrew Stanton,"Boston, Massachusetts, USA",5.120,/tRwWuo06aN9vuXAPaswMN42x2ii.jpg,NaN,NaN,NaN
3,0.0,[],"Robert ""Bob"" Peterson (born January 1961) is a...",1961-01-18,None,2.0,None,10.0,nm0677037,Acting,Bob Peterson,"Wooster, Ohio, USA",0.840,/1D5PtC97QwIks6xTjbJ1HNE8kbT.jpg,NaN,NaN,NaN
4,0.0,"[Dave Reynolds, David F. Reynolds]",Dave began his writing career in 1993 as one o...,1966-08-10,None,2.0,None,11.0,nm0721675,Writing,David Reynolds,None,1.176,/5iKtATPbLpv2lT7q9DPX2v2qPS1.jpg,NaN,NaN,NaN


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

##### 1) Clean up links dataframe

In [11]:
# Need to drop the tmdbId=NaN rows - only interested in movies that are actually in TMDB
links.dropna(axis=0, how='any', inplace=True)

# Can also drop the imdbId column; will be picking it up below in the proper format
links.drop(columns='imdbId', inplace=True)

##### 2) Clean up movies dataframe

In [12]:
# Will need to link the ids in movies dataframe to tmdbIds, so this is a good (and necessary)
#   place to add the links dataframe to the movies dataframe
movies = movies.merge(links, how='left', on='movieId')

# Retain only the titles in the movies dataframe that are actually confirmed (via API query) to be in TMDB (i.e. are in the people dataframe)
movies = movies[movies['tmdbId'].isin(people['id'])]
movies.reset_index(drop=True, inplace=True)

# Drop the handful of duplicates discovered in the movies df
movies = movies.drop_duplicates(subset=['tmdbId'], keep='first')
movies.reset_index(drop=True, inplace=True)

##### 3) Clean up ratings dataframe

In [13]:
# Drop NaN rows (if any)
ratings = ratings[~ratings.isna().any(axis=1)]

# Far more rows than we need here, and using all of them will needlessly slow 
#     down the ETL pipeline for this project; so, randomly sample down to 370,000 rows
ratings = ratings.sample(n=370000, random_state=999).reset_index(drop=True)

# Only include movies that are in the movies dataframe
ratings = ratings[ratings['movieId'].isin(movies['movieId'].unique())]

##### 4) Clean up tags dataframe

In [14]:
# Drop NaN rows (if any)
tags = tags[~tags.isna().any(axis=1)]  # Drop any NaN rows

# Far more rows than we need here, and using all of them will needlessly slow 
#     down the ETL pipeline for this project; so, randomly sample down to 370,000 rows
tags = tags.sample(n=370000, random_state=999).reset_index(drop=True)

# Convert all tags to lower case to make sure the same tag is not represented multiple ways
tags['tag'] = tags['tag'].str.lower()

# Only include movies that are in the movies dataframe
tags = tags[tags['movieId'].isin(movies['movieId'].unique())]

##### 5) Clean up people dataframe

In [15]:
# Fix a known TMDB query error discovered during data exploration: 
#   For TMDB movie id 125464, substitute correct actor 147518 for unknown actor 189129

people.loc[people['id'] == 125464.0]
people.loc[people['id'] == 125464.0, 'actor2_id'] = 147518.0
people.loc[people['id'] == 125464.0]

,id,actor1_id,actor2_id,actor3_id,actor4_id,actor5_id,director1_id,director2_id,director3_id,director4_id,director5_id,writer1_id,writer2_id,writer3_id,writer4_id,writer5_id
37806,125464,1081416.0,147518.0,209380.0,94492.0,1081420.0,68738.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Fix another known TMDB query error: 
#   For TMDB movie id 93195, substitute correct actor 133113 for unknown actor 1035856

people.loc[people['id'] == 93195.0]
people.loc[people['id'] == 93195.0, 'actor5_id'] = 133113.0
people.loc[people['id'] == 93195.0]

,id,actor1_id,actor2_id,actor3_id,actor4_id,actor5_id,director1_id,director2_id,director3_id,director4_id,director5_id,writer1_id,writer2_id,writer3_id,writer4_id,writer5_id
50604,93195,8791.0,80178.0,1035301.0,96481.0,133113.0,41968.0,NaN,NaN,NaN,NaN,41968.0,1035303.0,NaN,NaN,NaN


In [17]:
# Fix another known TMDB query error: 
#   For TMDB movie id 24128, there is no second writer credited, so set writer2_id to NaN

people.loc[people['id'] == 24128.0]
people.loc[people['id'] == 24128.0, 'writer2_id'] = np.nan
people.loc[people['id'] == 24128.0]

,id,actor1_id,actor2_id,actor3_id,actor4_id,actor5_id,director1_id,director2_id,director3_id,director4_id,director5_id,writer1_id,writer2_id,writer3_id,writer4_id,writer5_id
2748,24128,3078.0,91229.0,91230.0,91233.0,1194050.0,16294.0,NaN,NaN,NaN,NaN,16294.0,NaN,NaN,NaN,NaN


##### 6) Clean up details dataframe

In [18]:
# Drop unininteresting columns (NOTE: 'budget' and 'revenue' would have been interesting, but the
#   vast majority of titles queried do not include these two values, so we'll drop them)
details.drop(columns=['adult', 'backdrop_path', 'belongs_to_collection', 'homepage', 'budget', \
                      'original_title', 'overview', 'poster_path', 'production_companies', \
                      'production_countries', 'spoken_languages', 'status', 'tagline', 'revenue', \
                      'title', 'video', 'genres', 'original_language', 'popularity'], inplace=True)

##### 7) Clean up actors dataframe

In [19]:
# Drop unwanted/uninteresting columns from actors df
actors_df.drop(columns=['success', 'status_code', 'status_message', 'adult', 'also_known_as', 'biography', 'gender', \
                        'homepage', 'known_for_department', 'place_of_birth', 'popularity', 'profile_path'], \
              inplace=True)

##### 8) Clean up directors dataframe

In [20]:
# Drop unwanted/uninteresting columns from directors df
directors_df.drop(columns=['adult', 'also_known_as', 'biography', 'gender', 'homepage', \
                        'known_for_department', 'place_of_birth', 'popularity', 'profile_path'], \
              inplace=True)

##### 9) Clean up writers dataframe

In [21]:
# Drop unwanted/uninteresting columns from writers df
writers_df.drop(columns=['success', 'status_code', 'status_message', 'adult', 'also_known_as', 'biography', 'gender', \
                         'homepage', 'known_for_department', 'place_of_birth', 'popularity', 'profile_path'], \
              inplace=True)

# Drop the known single NaN (result of errant query)
writers_df = writers_df[~writers_df.isnull().all(axis=1)]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

NOTE: The query structure used below loads 2000 rows at a time, resulting in a significant time savings relative to inserting one row of data at a time. The methodology was adapted from comments and code suggested here: https://stackoverflow.com/questions/8134602/psycopg2-insert-multiple-rows-with-one-query

In [22]:
start = time.time()

##### 1) Create Tables

In [23]:
def create_database():
    """
    - Creates and connects to the sparkifydb
    - Returns the connection and cursor to sparkifydb
    """
    
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create movies database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS moviesdb")
    cur.execute("CREATE DATABASE moviesdb WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to movies database
    conn = psycopg2.connect("host=127.0.0.1 dbname=moviesdb user=student password=student")
    cur = conn.cursor()
    
    return cur, conn


def drop_tables(cur, conn):
    """
    Drops each table using the queries in `drop_table_queries` list.
    """
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    """
    Creates each table using the queries in `create_table_queries` list. 
    """
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    """
    - Drops (if exists) and Creates the sparkify database. 
    
    - Establishes connection with the sparkify database and gets
      cursor to it.  
    
    - Drops all the tables.  
    
    - Creates all tables needed. 
    
    - Finally, closes the connection. 
    """
    cur, conn = create_database()
    
    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

In [24]:
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.datetime64, psycopg2._psycopg.AsIs)

##### 2) Verify that tables were created

In [25]:
# Re-connect to the Movies Database
conn = psycopg2.connect("host=127.0.0.1 dbname=moviesdb user=student password=student")
conn.autocommit = True
cur = conn.cursor()

# Query to verify that the empty tables exist
try:
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('fact_user_actions',)
('dim_users',)
('dim_time',)
('dim_movies',)
('dim_actors',)
('dim_directors',)
('dim_writers',)


##### 3) dim_movies table ETL Pipeline

In [26]:
# Add the columns in the people df to the movies df
movies = movies.merge(people, how='left', left_on='tmdbId', right_on='id').drop(columns=['id'])

# Add the columns in the details df to the movies df
movies = movies.merge(details, how='left', left_on='tmdbId', right_on='id')
movies = movies.drop(columns='id')

# Covert timestamps to datetimes
movies['release_date'] = pd.to_datetime(movies['release_date'], yearfirst=True, errors = 'coerce')

# Rename and reorder columns to match the defined schema
movies.rename(columns={'movieId': 'movie_id', 'tmdbId': 'tmdb_id', 'vote_average': 'tmdb_rating_avg', \
                       'vote_count': 'tmdb_votes_tot'}, \
              inplace=True)
movies = movies[['movie_id', 'title', 'tmdb_id', 'imdb_id', 'genres', 'actor1_id', 'actor2_id', \
                 'actor3_id', 'actor4_id', 'actor5_id', 'director1_id', 'director2_id', \
                 'director3_id', 'director4_id', 'director5_id', 'writer1_id', 'writer2_id', \
                 'writer3_id', 'writer4_id', 'writer5_id', 'release_date', 'runtime', \
                 'tmdb_rating_avg', 'tmdb_votes_tot']]

# Make these ids strings to accommodate NaNs 
# for col in ['tmdb_id','actor1_id','actor2_id','actor3_id','actor4_id','actor5_id', \
#             'director1_id','director2_id','director3_id','director4_id','director5_id', \
#             'writer1_id','writer2_id','writer3_id','writer4_id','writer5_id']:
#     movies[col] = movies[col].astype(float).apply(str)
# Change NaNs to None to accomodate postgreSQL
movies.replace({np.NaN: None}, inplace=True)

# Load the dim_movies table
print('\n Loading dim_movies table...\n')
for i in range(math.ceil(movies.shape[0] / 2000)):
    tup = movies.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, movies_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_movies table...


Time Elapsed So Far: 0.2 mins



##### 4) dim_actors ETL Pipeline

In [27]:
# Covert timestamps to datetimes
actors_df['birthday'] = pd.to_datetime(actors_df['birthday'], yearfirst=True, errors = 'coerce')
actors_df['deathday'] = pd.to_datetime(actors_df['deathday'], yearfirst=True, errors = 'coerce')

# Rename the column ids & reorder to match the specified schema
actors_df.rename(columns={'id': 'actor_id'}, inplace=True)
actors_df = actors_df[['actor_id', 'name', 'imdb_id', 'birthday', 'deathday']]

# Clear known NaN rows
actors_df = actors_df[~actors_df['actor_id'].isna()]

# Change NaNs to None to accommodate postgreSQL
actors_df.replace({np.NaN: None}, inplace=True)

# Load the dim_actors table
print('\n Loading dim_actors table...\n')
for i in range(math.ceil(actors_df.shape[0] / 2000)):
    tup = actors_df.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, actors_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_actors table...


Time Elapsed So Far: 0.3 mins



##### 5) dim_directors ETL Pipeline

In [28]:
# Covert timestamps to datetimes
directors_df['birthday'] = pd.to_datetime(directors_df['birthday'], yearfirst=True, errors = 'coerce')
directors_df['deathday'] = pd.to_datetime(directors_df['deathday'], yearfirst=True, errors = 'coerce')

# Rename the column ids & reorder to match the specified schema
directors_df.rename(columns={'id': 'director_id'}, inplace=True)
directors_df = directors_df[['director_id', 'name', 'imdb_id', 'birthday', 'deathday']]

# Change NaNs to None to accommodate postgreSQL
directors_df.replace({np.NaN: None}, inplace=True)

# Load the dim_directors table
print('\n Loading dim_directors table...\n')
for i in range(math.ceil(directors_df.shape[0] / 2000)):
    tup = directors_df.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, directors_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_directors table...


Time Elapsed So Far: 0.4 mins



##### 6) dim_writers ETL Pipeline

In [29]:
# Covert timestamps to datetimes
writers_df['birthday'] = pd.to_datetime(writers_df['birthday'], yearfirst=True, errors = 'coerce')
writers_df['deathday'] = pd.to_datetime(writers_df['deathday'], yearfirst=True, errors = 'coerce')

# Rename the column ids & reorder to match the specified schema
writers_df.rename(columns={'id': 'writer_id'}, inplace=True)
writers_df = writers_df[['writer_id', 'name', 'imdb_id', 'birthday', 'deathday']]

# Change NaNs to None to accommodate postgreSQL
writers_df.replace({np.NaN: None}, inplace=True)

# Load the dim_writers table
print('\n Loading dim_writers table...\n')
for i in range(math.ceil(writers_df.shape[0] / 2000)):
    tup = writers_df.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, writers_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_writers table...


Time Elapsed So Far: 0.4 mins



##### 7) fact_user_actions ETL Pipline

In [30]:
# Create the user_actions dataframe using ratings and tags dataframes; 
#    action can be 'rate' or 'tag'; 
#    action_val is a tag or 0-5 rating (represented as a string)
user_actions = ratings.copy(deep=True)
user_actions['action'] = 'rate'
user_actions['action_val'] = user_actions['rating'].apply(str)
user_actions.drop(columns=['rating'], inplace=True)
user_actions = user_actions[['userId', 'timestamp', 'movieId', 'action', 'action_val']]
user_actions2 = tags.copy(deep=True)
user_actions2['action'] = 'tag'
user_actions2.rename(columns={'tag': 'action_val'}, inplace=True)
user_actions2 = user_actions2[['userId', 'timestamp', 'movieId', 'action', 'action_val']]
user_actions = pd.concat([user_actions, user_actions2], axis=0)
user_actions.reset_index(drop=True, inplace=True)

# Convert timestamps to datetimes
user_actions.loc[:, 'timestamp'] = \
                user_actions.loc[:, 'timestamp'] \
                            .apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
user_actions.rename(columns={'userId': 'user_id', 'timestamp': 'action_time', 'movieId': 'movie_id'}, inplace=True)

# Load the fact_user_actions table
print('\n Loading fact_user_actions table...\n')
for i in range(math.ceil(user_actions.shape[0] / 2000)):
    tup = user_actions.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, user_actions_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading fact_user_actions table...


Time Elapsed So Far: 1.1 mins



##### 8) dim_users ETL Pipeline

In [31]:
# This code computes 'num_ratings', 'num_tags' and 'avg_rating' per user represented in fact_user_actions table

users_df = pd.DataFrame(data={'user_id': np.sort(user_actions['user_id'].unique())})
user_actions['rated'] = user_actions['action'].apply(lambda x: 1 if (x == 'rate') else 0)
user_actions['tagged'] = user_actions['action'].apply(lambda x: 1 if (x == 'tag') else 0)
user_actions['rating_val'] = user_actions[['action','action_val']] \
                            .apply(lambda x: np.nan if (x['action'] == 'tag') else float(x['action_val']), \
                                  axis=1)
user_actions['num_ratings'] = user_actions.groupby('user_id')['rated'].transform(np.sum)
user_actions['num_tags'] = user_actions.groupby('user_id')['tagged'].transform(np.sum)
user_actions['avg_rating'] = user_actions.groupby('user_id')['rating_val'].transform(np.mean)
temp = user_actions.groupby('user_id').first().reset_index()
users_df['num_ratings'] = temp['num_ratings']
users_df['num_tags'] = temp['num_tags']
users_df['avg_rating'] = temp['avg_rating']
user_actions.drop(columns=['rated', 'tagged', 'num_ratings', 'num_tags', 'rating_val', 'avg_rating'], \
                  inplace=True)

# Change NaNs to None to accomodate postgreSQL
users_df.replace({np.NaN: None}, inplace=True)

# Load the dim_users table
print('\n Loading dim_users table...\n')
for i in range(math.ceil(users_df.shape[0] / 2000)):
    tup = users_df.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, users_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTime Elapsed So Far: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_users table...


Time Elapsed So Far: 1.5 mins



##### 9) dim_time ETL Pipeline

In [32]:
# All event times appearing in fact_user_actions table (with duplicate rows dropped)

t = user_actions['action_time'].copy(deep=True)
t = t.drop_duplicates().reset_index(drop=True)
time_data = pd.concat([t, pd.to_datetime(t).dt.hour, pd.to_datetime(t).dt.day, pd.to_datetime(t).dt.weekofyear, \
           pd.to_datetime(t).dt.month, pd.to_datetime(t).dt.year], axis=1).values.tolist()
column_labels = ['action_time', 'hour', 'day', 'week', 'month', 'year']
time_df = pd.DataFrame(data=time_data, columns=column_labels)
# time_df['action_time'] = pd.to_datetime(time_df[['action_time']] \
#                                         .apply(lambda x: x[0].timestamp()*1e9, axis=1).astype(int), yearfirst=True, errors = 'coerce')

# Load the dim_time table
print('\n Loading dim_time table...\n')
for i in range(math.ceil(time_df.shape[0] / 2000)):
    tup = time_df.iloc[ 2000*i : 2000*(i+1), :]
    tup = list(tup.to_records(index=False))
    try:
        execute_values(cur, time_table_insert, tup) 
    except psycopg2.Error as e:
        print(e)
        break
        
print('\nTotal elapsed time to load all tables: {} mins\n'.format(str(round((time.time() - start)/60, 1))))


 Loading dim_time table...


Total elapsed time to load all tables: 2.4 mins



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

##### 1) Query dim_movies table to verify it contains data in expected form:

In [33]:
try:
    cur.execute("SELECT * FROM dim_movies")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(1, 'Toy Story (1995)', 862, 'tt0114709', 'Adventure|Animation|Children|Comedy|Fantasy', 31, 12898, 7167, 12899, 12900, 7879, None, None, None, None, 7, 12891, 12892, 12893, None, datetime.date(1995, 10, 30), 81.0, 7.9, 14111)
(2, 'Jumanji (1995)', 8844, 'tt0113497', 'Adventure|Children|Fantasy', 2157, 205, 145151, 5149, 8537, 4945, None, None, None, None, 876, 56520, 56521, None, None, datetime.date(1995, 12, 15), 104.0, 7.2, 8260)
(3, 'Grumpier Old Men (1995)', 15602, 'tt0113228', 'Comedy|Romance', 6837, 3151, 13567, 16757, 589, 26502, None, None, None, None, 16837, None, None, None, None, datetime.date(1995, 12, 22), 101.0, 6.5, 251)
(4, 'Waiting to Exhale (1995)', 31357, 'tt0114885', 'Comedy|Drama|Romance', 8851, 9780, 18284, 51359, 66804, 2178, None, None, None, None, 5144, 111118, None, None, None, datetime.date(1995, 12, 22), 127.0, 6.3, 95)
(5, 'Father of the Bride Part II (1995)', 11862, 'tt0113041', 'Comedy', 67773, 3092, 519, 70696, 59222, 56106, None, None, None, None, 1769

##### 2) Query dim_movies table verify number of expected rows (should be 57,160):

In [34]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_movies")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(57160,)


##### 3) Query dim_actors table to verify it contains data in expected form:

In [35]:
try:
    cur.execute("SELECT * FROM dim_actors")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(1, 'George Lucas', 'nm0000184', datetime.date(1944, 5, 14), None)
(2, 'Mark Hamill', 'nm0000434', datetime.date(1951, 9, 25), None)
(3, 'Harrison Ford', 'nm0000148', datetime.date(1942, 7, 13), None)
(4, 'Carrie Fisher', 'nm0000402', datetime.date(1956, 10, 21), datetime.date(2016, 12, 27))
(5, 'Peter Cushing', 'nm0001088', datetime.date(1913, 5, 26), datetime.date(1994, 8, 11))
(6, 'Anthony Daniels', 'nm0000355', datetime.date(1946, 2, 21), None)
(10, 'Bob Peterson', 'nm0677037', datetime.date(1961, 1, 18), None)
(12, 'Alexander Gould', 'nm1071252', datetime.date(1994, 5, 4), None)
(13, 'Albert Brooks', 'nm0000983', datetime.date(1947, 7, 22), None)
(14, 'Ellen DeGeneres', 'nm0001122', datetime.date(1958, 1, 26), None)


##### 4) Query dim_actors table verify number of expected rows (should be 93,344):

In [36]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_actors")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(93344,)


##### 5) Query dim_directors table to verify it contains data in expected form:

In [37]:
try:
    cur.execute("SELECT * FROM dim_directors")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(1, 'George Lucas', 'nm0000184', datetime.date(1944, 5, 14), None)
(2, 'Mark Hamill', 'nm0000434', datetime.date(1951, 9, 25), None)
(7, 'Andrew Stanton', 'nm0004056', datetime.date(1965, 12, 3), None)
(8, 'Lee Unkrich', 'nm0881279', datetime.date(1967, 8, 8), None)
(13, 'Albert Brooks', 'nm0000983', datetime.date(1947, 7, 22), None)
(24, 'Robert Zemeckis', 'nm0000709', datetime.date(1951, 5, 14), None)
(31, 'Tom Hanks', 'nm0000158', datetime.date(1956, 7, 9), None)
(33, 'Gary Sinise', 'nm0000641', datetime.date(1955, 3, 17), None)
(35, 'Sally Field', 'nm0000398', datetime.date(1946, 11, 6), None)
(39, 'Sam Mendes', 'nm0005222', datetime.date(1965, 8, 1), None)


##### 6) Query dim_directors table verify number of expected rows (should be 24,223):

In [38]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_directors")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(24223,)


##### 7) Query dim_writers table to verify it contains data in expected form:

In [39]:
try:
    cur.execute("SELECT * FROM dim_writers")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned    
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(1, 'George Lucas', 'nm0000184', datetime.date(1944, 5, 14), None)
(4, 'Carrie Fisher', 'nm0000402', datetime.date(1956, 10, 21), datetime.date(2016, 12, 27))
(7, 'Andrew Stanton', 'nm0004056', datetime.date(1965, 12, 3), None)
(10, 'Bob Peterson', 'nm0677037', datetime.date(1961, 1, 18), None)
(11, 'David Reynolds', 'nm0721675', datetime.date(1966, 8, 10), None)
(13, 'Albert Brooks', 'nm0000983', datetime.date(1947, 7, 22), None)
(14, 'Ellen DeGeneres', 'nm0001122', datetime.date(1958, 1, 26), None)
(22, 'Barry Humphries', 'nm0402032', datetime.date(1934, 2, 17), None)
(24, 'Robert Zemeckis', 'nm0000709', datetime.date(1951, 5, 14), None)
(27, 'Eric Roth', 'nm0744839', datetime.date(1945, 3, 22), None)


##### 8) Query dim_writers table verify number of expected rows (should be 35,584):

In [40]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_writers")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(35584,)


##### 9) Query fact_user_actions table to verify it contains data in expected form:

In [41]:
try:
    cur.execute("SELECT * FROM fact_user_actions")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned 
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(1, 20544, datetime.datetime(2003, 10, 10, 16, 0, 44), 509, 'rate', '3.5')
(2, 11949, datetime.datetime(2004, 3, 23, 6, 4, 49), 6953, 'rate', '4.5')
(3, 9986, datetime.datetime(2016, 2, 26, 16, 27, 28), 153070, 'rate', '3.0')
(4, 9990, datetime.datetime(2000, 11, 4, 21, 56, 51), 2866, 'rate', '3.0')
(5, 53264, datetime.datetime(2015, 11, 7, 23, 3, 24), 61248, 'rate', '3.5')
(6, 84531, datetime.datetime(2007, 11, 11, 19, 23, 11), 1064, 'rate', '1.5')
(7, 77412, datetime.datetime(2015, 1, 24, 20, 36, 51), 163, 'rate', '3.0')
(8, 70002, datetime.datetime(2008, 1, 27, 3, 13, 55), 48304, 'rate', '0.5')
(9, 46682, datetime.datetime(2005, 4, 2, 15, 46, 16), 150, 'rate', '4.0')
(10, 91209, datetime.datetime(2018, 3, 8, 12, 48, 51), 5481, 'rate', '4.0')


##### 10) Query fact_user_actions table verify number of expected rows (should be 736,680):

In [42]:
try:
    cur.execute("SELECT COUNT(*) FROM fact_user_actions")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(736680,)


##### 11) Query dim_users table to verify it contains data in expected form:

In [43]:
try:
    cur.execute("SELECT * FROM dim_users")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned     
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(2, 2, 0, 3.5)
(4, 25, 0, 3.04)
(6, 1, 0, 4.0)
(7, 1, 0, 3.5)
(10, 5, 0, 4.0)
(14, 5, 3, 3.9)
(15, 8, 0, 4.1875)
(16, 2, 0, 5.0)
(18, 3, 0, 3.0)
(19, 10, 0, 3.7)


##### 12) Query dim_users table verify number of expected rows (should be 80,478):

In [44]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_users")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(80478,)


##### 13) Query dim_time table to verify it contains data in expected form:

In [45]:
try:
    cur.execute("SELECT * FROM dim_time")
except psycopg2.Error as e:
    print(e)

# Print first 10 rows returned
row = cur.fetchone()
i = 0
while i < 10:
    print(row)
    row = cur.fetchone()
    i += 1

(datetime.datetime(2003, 10, 10, 16, 0, 44), 16, 10, 41, 10, 2003)
(datetime.datetime(2004, 3, 23, 6, 4, 49), 6, 23, 13, 3, 2004)
(datetime.datetime(2016, 2, 26, 16, 27, 28), 16, 26, 8, 2, 2016)
(datetime.datetime(2000, 11, 4, 21, 56, 51), 21, 4, 44, 11, 2000)
(datetime.datetime(2015, 11, 7, 23, 3, 24), 23, 7, 45, 11, 2015)
(datetime.datetime(2007, 11, 11, 19, 23, 11), 19, 11, 45, 11, 2007)
(datetime.datetime(2015, 1, 24, 20, 36, 51), 20, 24, 4, 1, 2015)
(datetime.datetime(2008, 1, 27, 3, 13, 55), 3, 27, 4, 1, 2008)
(datetime.datetime(2005, 4, 2, 15, 46, 16), 15, 2, 13, 4, 2005)
(datetime.datetime(2018, 3, 8, 12, 48, 51), 12, 8, 10, 3, 2018)


##### 14) Query dim_time table verify number of expected rows (should be 686,443):

In [46]:
try:
    cur.execute("SELECT COUNT(*) FROM dim_time")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(686443,)


In [47]:
# Try finding how many users have rated more than 100 movies

try:
    cur.execute("SELECT COUNT(*) FROM dim_users WHERE num_ratings > 100")
except psycopg2.Error as e:
    print(e)
    
finished = False
i = 0
row = cur.fetchone()
print(row)

(52,)


In [48]:
# Try finding users who have rated more than 100 movies

try:
    cur.execute("SELECT * FROM dim_users WHERE num_ratings > 100;")
except psycopg2.Error as e:
    print(e)
    
finished = False
i = 1
row = cur.fetchone()
while not finished:
    print(row)
    row = cur.fetchone()
    if (row == None) or i == 52:
        finished = True
    else:
        i += 1

(2025, 110, 0, 3.68636363636364)
(2294, 107, 79, 3.28504672897196)
(4796, 168, 0, 2.9672619047619)
(5323, 106, 1, 3.45754716981132)
(5747, 116, 0, 3.00431034482759)
(7705, 151, 0, 3.13907284768212)
(11109, 133, 612, 3.30827067669173)
(14320, 108, 5, 2.40277777777778)
(15041, 105, 0, 2.69047619047619)
(16225, 109, 0, 2.89908256880734)
(19007, 101, 2711, 2.84653465346535)
(19309, 106, 1, 4.34433962264151)
(19596, 116, 0, 2.98275862068966)
(19924, 118, 15, 3.14406779661017)
(23026, 107, 8, 3.83177570093458)
(23649, 112, 0, 3.27232142857143)
(29647, 131, 77, 3.55725190839695)
(34504, 108, 0, 3.07407407407407)
(35453, 105, 4, 2.76666666666667)
(37046, 157, 0, 3.24203821656051)
(42746, 102, 0, 2.57352941176471)
(48470, 181, 1, 2.43370165745856)
(52278, 103, 0, 2.49514563106796)
(57392, 118, 5, 3.28813559322034)
(59455, 107, 0, 1.4392523364486)
(60950, 174, 0, 3.0948275862069)
(61614, 149, 0, 3.52348993288591)
(63783, 220, 1, 3.61136363636364)
(64180, 115, 0, 3.57391304347826)
(65670, 101, 1,

In [49]:
# Most movies rated

try:
    cur.execute("SELECT MAX(num_ratings) FROM dim_users;")
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()
print(row)

(233,)


In [50]:
# Highest IMDB Rated Movie

try:
    cur.execute("SELECT * FROM dim_movies WHERE tmdb_rating_avg = (SELECT MAX(tmdb_rating_avg) FROM dim_movies);")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(2223, "Farmer's Wife, The (1928)", 143750, 'tt2140519', 'Comedy', 1120400, 11855, 976738, 1120401, None, 192852, None, None, None, None, 192852, None, None, None, None, datetime.date(2012, 6, 20), 18.0, 10.0, 1)


In [51]:
# Highest IMDB Rated Movie with non-trivial number of ratings (i.e. > 1000 votes)

try:
    cur.execute("SELECT * FROM dim_movies WHERE tmdb_rating_avg = (SELECT MAX(tmdb_rating_avg) FROM dim_movies WHERE tmdb_votes_tot > 1000);")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(318, 'Shawshank Redemption, The (1994)', 278, 'tt0111161', 'Crime|Drama', 504, 192, 4029, 6573, 6574, 4027, None, None, None, None, 4027, None, None, None, None, datetime.date(1994, 9, 23), 142.0, 8.7, 19136)


In [52]:
# Top 2 Actors, Director and Writer for 'Shawshank Redemption' 

try:
    cur.execute('''
        SELECT da1.name, da2.name, dd.name, dw.name
        FROM dim_actors da1 INNER JOIN dim_movies m ON da1.actor_id = m.actor1_id
        INNER JOIN dim_actors da2 ON da2.actor_id = m.actor2_id
        INNER JOIN dim_directors dd ON dd.director_id = m.director1_id
        INNER JOIN dim_writers dw ON dw.writer_id = m.writer1_id
        WHERE m.title = 'Shawshank Redemption, The (1994)';
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

('Tim Robbins', 'Morgan Freeman', 'Frank Darabont', 'Frank Darabont')


In [53]:
# Avg User Rating and Number of User Votes for 'The Shawshank Redemption' (movie_id = 318)

try:
    cur.execute('''
        SELECT AVG(action_val::numeric::float), COUNT(*)
        FROM fact_user_actions
        WHERE movie_id = 318 AND action = 'rate';
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(4.42490421455939, 1305)


In [54]:
# Number of distinct user tags for 'The Shawshank Redemption' (movie_id = 318)

try:
    cur.execute('''
        SELECT COUNT(DISTINCT action_val) FROM fact_user_actions
        WHERE movie_id = 318 AND action = 'tag';
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(246,)


In [55]:
# List all distinct user tags for 'The Shawshank Redemption' (movie_id = 318)

try:
    cur.execute('''
        SELECT DISTINCT action_val FROM fact_user_actions
        WHERE movie_id = 318 AND action = 'tag';
    ''')
except psycopg2.Error as e:
    print(e)

finished = False
i = 1
row = cur.fetchone()
while not finished:
    print(row)
    row = cur.fetchone()
    if (row == None) or i == 246:
        finished = True
    else:
        i += 1

('obvious',)
('téa leoni does not star in this movie',)
('adventure',)
('oscar nominee: supporting actor',)
('psychology',)
('awesome!',)
('stephan king',)
('underdog',)
('uplifting',)
('corruption',)
('fun',)
('5 stars',)
('mistery',)
('based on a book',)
('very moving',)
('literature',)
('great cast',)
('visually stunning',)
('heart-wrenching',)
('justice',)
('not as good as one might think',)
('repair un injustice',)
('carcel',)
('moral ambiguity',)
('antihero',)
('inspirational',)
('compassionate',)
('feel-good',)
('excellent and worth while.',)
('emotional',)
('best movie ever',)
('realism',)
('clv',)
('5\\+++',)
('favorite',)
('existentialism',)
('jail',)
('clever',)
('freedom',)
('deceit',)
('sentimental',)
('soppy',)
('abcd',)
('believe',)
('philosophy',)
('rated r',)
('best-movie',)
('top10',)
('bfi modern classic',)
('bibliothek',)
('wrongly convicted',)
('adapted from:short story',)
('prison drama',)
('100 greatest movies',)
('steven king',)
('frank darabont',)
('bankchiitin

In [56]:
# Earliest user event ever (from dim_time)

try:
    cur.execute('''
        SELECT * FROM dim_time 
        WHERE action_time = (SELECT MIN(action_time) FROM dim_time);
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(datetime.datetime(1996, 2, 2, 12, 36, 52), 12, 2, 5, 2, 1996)


In [59]:
# User action(s) and movie title associated with the earliest user event ever (from dim_time)

try:
    cur.execute('''
        SELECT f.user_id, f.action_time, f.action, f.action_val, dm.title 
        FROM fact_user_actions f INNER JOIN dim_movies dm
        ON f.movie_id = dm.movie_id
        WHERE action_time = (SELECT MIN(action_time) FROM dim_time);
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(32913, datetime.datetime(1996, 2, 2, 12, 36, 52), 'rate', '4.0', 'Dangerous Minds (1995)')


In [63]:
# Latest user event ever (from dim_time)

try:
    cur.execute('''
        SELECT * FROM dim_time 
        WHERE action_time = (SELECT MAX(action_time) FROM dim_time);
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(datetime.datetime(2018, 9, 26, 6, 55, 49), 6, 26, 39, 9, 2018)


In [60]:
# User action(s) and movie title associated with the latest user event ever (from dim_time)

try:
    cur.execute('''
        SELECT f.user_id, f.action_time, f.action, f.action_val, dm.title 
        FROM fact_user_actions f INNER JOIN dim_movies dm
        ON f.movie_id = dm.movie_id 
        WHERE action_time = (SELECT MAX(action_time) FROM dim_time);
    ''')
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
print(row)

(243095, datetime.datetime(2018, 9, 26, 6, 55, 49), 'tag', 'good soundtrack', 'Ready Player One')


In [61]:
# Close the database connection
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.